<h1>Yahoo Finance Ticker Statistics Page Scraper</h1>
<h3>This script takes a list of tickers and scraps its info from the yahoo finance statistics page.</h3>

<a href = "https://finance.yahoo.com/quote/TSLA/key-statistics?p=TSLA"><img src="sample.JPG"></a>

In [47]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
import time

In [48]:
tickers = ['CRWD',
           'TEAM',
           'ZS',
           'NET',
           'SNOW',
           'HUBS',
           'U',
           'DDOG',
           'DOCN',
           'ZM',
           'SEMR',
           'MDB',
           'SHOP']

In [49]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
soups=[]

for ticker in tickers:
    url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    resp = requests.get(url, headers = headers)
    print(ticker, resp.status_code)
    #time.sleep(1)
    soup = BeautifulSoup(resp.text, "html.parser")
    soups.append(soup)

CRWD 200
TEAM 200
ZS 200
NET 200
SNOW 200
HUBS 200
U 200
DDOG 200
DOCN 200
ZM 200
SEMR 200
MDB 200
SHOP 200


In [50]:
tds = soup.find_all("td")

n= 0
titles= []

for td in tds:
    n+=1
    if n%2 == 0:
        continue
    else:
        text = td.text.strip()
        titles.append(text)
    
all_stats_df = pd.DataFrame({"Metrics":titles})
all_stats_df.head()

,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)


In [51]:
all_stats_df['Metrics'].replace(regex={r'[0-9]$': ''}, inplace = True) 
#removes the annotations appearing at the end of rows

display(all_stats_df.iloc[23:28])
all_stats_df.iloc[23:28,0].replace(regex={r'(\(.+\))': ''}, inplace = True) 
#remove the dates under rows 23-27

all_stats_df

,Metrics
23,"Shares Short (Aug 30, 2022)"
24,"Short Ratio (Aug 30, 2022)"
25,"Short % of Float (Aug 30, 2022)"
26,"Short % of Shares Outstanding (Aug 30, 2022)"
27,"Shares Short (prior month Jul 28, 2022)"


,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)
5,Price/Sales (ttm)
6,Price/Book (mrq)
7,Enterprise Value/Revenue
8,Enterprise Value/EBITDA
9,Beta (5Y Monthly)


In [52]:
for soup, ticker in zip(soups, tickers):
    tds = soup.find_all("td")
    n= 0
    stats= []

    for td in tds:
        n+=1
        if n%2 != 0:
            continue
        else:
            text = td.text.strip()
            stats.append(text)
    all_stats_df[f'{ticker}'] = stats
    
all_stats_df

,Metrics,CRWD,TEAM,ZS,NET,SNOW,HUBS,U,DDOG,DOCN,ZM,SEMR,MDB,SHOP
0,Market Cap (intraday),41.74B,61.93B,24.97B,19.29B,62.88B,14.00B,10.68B,28.88B,4.03B,23.42B,1.63B,16.04B,40.25B
1,Enterprise Value,40.20B,61.78B,24.28B,19.22B,59.17B,13.50B,10.74B,27.98B,4.33B,18.00B,1.44B,15.44B,34.48B
2,Trailing P/E,N/A,N/A,N/A,N/A,N/A,N/A,N/A,9.12k,N/A,24.28,N/A,N/A,N/A
3,Forward P/E,149.25,147.06,151.52,625.00,1.25k,104.17,N/A,86.96,37.59,21.98,N/A,N/A,5.00k
4,PEG Ratio (5 yr expected),3.69,4.84,3.24,N/A,6.35,3.02,N/A,1.49,N/A,11.10,N/A,N/A,N/A
5,Price/Sales (ttm),22.47,21.95,22.73,23.28,37.33,9.06,8.48,21.93,N/A,5.61,7.19,14.53,8.05
6,Price/Book (mrq),34.06,184.52,43.55,33.24,11.59,16.07,4.68,23.76,N/A,3.91,7.47,24.01,4.62
7,Enterprise Value/Revenue,21.92,N/A,22.26,23.65,36.12,8.85,8.81,20.49,8.79,4.19,6.48,14.26,6.89
8,Enterprise Value/EBITDA,-1.03k,-138.90,-86.60,-119.97,-88.54,-570.68,-20.82,440.70,67.46,19.71,-287.19,-48.68,-17.87
9,Beta (5Y Monthly),1.27,0.94,1.02,1.10,N/A,1.55,N/A,1.13,N/A,-0.49,N/A,0.94,1.82


In [53]:
col_list = all_stats_df['Metrics']
t_all_stats_df = all_stats_df.transpose()
t_all_stats_df.columns = col_list
t_all_stats_df.drop(index='Metrics', inplace= True)
t_all_stats_df.reset_index(inplace=True)
t_all_stats_df.rename({'index' : 'Ticker'}, axis = 1, inplace=True)

t_all_stats_df

Metrics,Ticker,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,CRWD,41.74B,40.20B,N/A,149.25,3.69,22.47,34.06,21.92,-1.03k,...,-0.79,N/A,2.32B,9.94,771.92M,62.08,1.80,5.25,743.64M,656.29M
1,TEAM,61.93B,61.78B,N/A,147.06,4.84,21.95,184.52,N/A,-138.90,...,-2.86,N/A,1.47B,5.77,1.31B,391.68,1.19,1.32,883.5M,253.75M
2,ZS,24.97B,24.28B,N/A,151.52,3.24,22.73,43.55,22.26,-86.60,...,-2.67,N/A,1.73B,12.1,1.05B,182.40,1.99,4.01,321.91M,480.98M
3,NET,19.29B,19.22B,N/A,625.00,N/A,23.28,33.24,23.65,-119.97,...,-0.83,N/A,1.64B,5.01,1.57B,270.03,5.45,1.77,36.48M,91.88M
4,SNOW,62.88B,59.17B,N/A,1.25k,6.35,37.33,11.59,36.12,-88.54,...,-2.10,N/A,3.95B,12.35,241.76M,4.45,3.21,16.96,343.48M,606.77M
5,HUBS,14.00B,13.50B,N/A,104.17,3.02,9.06,16.07,8.85,-570.68,...,-1.36,N/A,1.25B,25.95,746.21M,85.63,2.31,18.19,260.98M,225.78M
6,U,10.68B,10.74B,N/A,N/A,N/A,8.48,4.68,8.81,-20.82,...,-2.10,N/A,1.75B,5.88,1.82B,79.59,3.44,7.66,62.54M,168.05M
7,DDOG,28.88B,27.98B,9.12k,86.96,1.49,21.93,23.76,20.49,440.70,...,0.01,N/A,1.7B,5.38,811.29M,66.77,3.31,3.84,403.52M,353.84M
8,DOCN,4.03B,4.33B,N/A,37.59,N/A,N/A,N/A,8.79,67.46,...,-0.32,N/A,1.17B,12.03,1.47B,"3,037.97",16.59,0.50,148.57M,56.85M
9,ZM,23.42B,18.00B,24.28,21.98,11.10,5.61,3.91,4.19,19.71,...,3.30,-85.60%,5.52B,18.55,97.94M,1.63,3.44,20.10,1.39B,1.45B


In [147]:
import re

def cleaning(x):
    cleaned = x
    if re.match('(^\d+.\d+[B])', x) or re.match('(^\-\d+.\d+[B])', x):
        cleaned = re.sub("[^\-0-9]", "", x)
        cleaned = cleaned + "000000000"
    if re.match('(^\d+.\d+[M])', x) or re.match('(^\-\d+.\d+[M])', x):
        cleaned = re.sub("[^\-0-9]", "", x)
        cleaned = cleaned + "000000"
    if re.match('(^\d+.\d+[k])', x) or re.match('(^\-\d+.\d+[k])', x):
        cleaned = re.sub("[^\-0-9]", "", x)
        cleaned = cleaned + "000"
    cleaned = re.sub("[%]", "", cleaned)
    if x == "N/A":
        cleaned = 0
    return cleaned


cleaned_df = t_all_stats_df.applymap(cleaning)
cleaned_df

Metrics,Ticker,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,CRWD,4174000000000,4020000000000,0,149.25,3.69,22.47,34.06,21.92,-103000,...,-0.79,0,232000000000,9.94,77192000000,62.08,1.80,5.25,74364000000,65629000000
1,TEAM,6193000000000,6178000000000,0,147.06,4.84,21.95,184.52,0,-138.90,...,-2.86,0,147000000000,5.77,131000000000,391.68,1.19,1.32,8835000000,25375000000
2,ZS,2497000000000,2428000000000,0,151.52,3.24,22.73,43.55,22.26,-86.60,...,-2.67,0,173000000000,12.1,105000000000,182.40,1.99,4.01,32191000000,48098000000
3,NET,1929000000000,1922000000000,0,625.00,0,23.28,33.24,23.65,-119.97,...,-0.83,0,164000000000,5.01,157000000000,270.03,5.45,1.77,3648000000,9188000000
4,SNOW,6288000000000,5917000000000,0,125000,6.35,37.33,11.59,36.12,-88.54,...,-2.10,0,395000000000,12.35,24176000000,4.45,3.21,16.96,34348000000,60677000000
5,HUBS,1400000000000,1350000000000,0,104.17,3.02,9.06,16.07,8.85,-570.68,...,-1.36,0,125000000000,25.95,74621000000,85.63,2.31,18.19,26098000000,22578000000
6,U,1068000000000,1074000000000,0,0,0,8.48,4.68,8.81,-20.82,...,-2.10,0,175000000000,5.88,182000000000,79.59,3.44,7.66,6254000000,16805000000
7,DDOG,2888000000000,2798000000000,912000,86.96,1.49,21.93,23.76,20.49,440.70,...,0.01,0,17000000000,5.38,81129000000,66.77,3.31,3.84,40352000000,35384000000
8,DOCN,403000000000,433000000000,0,37.59,0,0,0,8.79,67.46,...,-0.32,0,117000000000,12.03,147000000000,"3,037.97",16.59,0.50,14857000000,5685000000
9,ZM,2342000000000,1800000000000,24.28,21.98,11.10,5.61,3.91,4.19,19.71,...,3.30,-85.60,552000000000,18.55,9794000000,1.63,3.44,20.10,139000000000,145000000000


In [54]:
current_datetime = date.today().isoformat()
t_all_stats_df.to_csv(f'saas_{current_datetime}.csv')
print(f'Saved to file: saas_{current_datetime}.csv')

Saved to file: saas_2022-09-18.csv
